In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from modules.benchmarks import MultiPeriodMeanVariance
import cvxpy as cp
import matplotlib.pyplot as plt
import seaborn as sns
import json
sns.set()

In [2]:
config_file_name = "config.json"
with open(config_file_name, "r") as jsonfile:
    config_data = json.load(jsonfile)

In [ ]:
cac40 = config_data["tickers"]["Europe"]["CAC_40"]
NUM_ASSETS = len(cac40)
len(cac40)

In [ ]:
start_date = config_data["timeframe"]["cac40"]["start_date"]
end_date = config_data["timeframe"]["cac40"]["end_date"]
data_interval = config_data["data_interval"]
test_ratio = config_data["train_test_ratio"]
random_seed = config_data["random_state_seed"]

In [ ]:
dataset = yf.download(cac40, 
                      start=start_date, 
                      end=end_date,
                      interval=data_interval)['Adj Close']
dataset.to_csv("data/raw/cac40_stocks_price.csv")
dataset_price_change = dataset.pct_change().fillna(0)
dataset_return = dataset/dataset.shift(1).fillna(1)

In [ ]:
NUM_ASSETS = len(dataset.columns.unique().tolist())

In [ ]:
_, test_df = train_test_split(dataset_price_change, test_size=test_ratio, shuffle=False, random_state=random_seed)
len(test_df)

In [ ]:
lookback_window = config_data["lookback_window"] # W
multi_step_horizon = config_data["multi_step_horizon"] # T
lookback_window, multi_step_horizon

In [ ]:
test_df_cumulative_return = dataset_return.loc[test_df.index,:]

In [ ]:
X_test = []
for i in range(len(test_df.values)-lookback_window):
    Sigmas = []
    mu = []
    for j in range(multi_step_horizon):
        cov_matrix_j = test_df.iloc[i+j:i+j+lookback_window-multi_step_horizon].cov().values
        mu_j = test_df.iloc[i+j:i+j+lookback_window-multi_step_horizon].mean(0).values
        Sigmas.append(cov_matrix_j)
        mu.append(mu_j)
    hist = test_df_cumulative_return.iloc[i:i+lookback_window]
    pred = test_df_cumulative_return.iloc[i+lookback_window:i+lookback_window+multi_step_horizon]
    X_test.append([hist, np.stack(mu), np.stack(Sigmas), pred])

len(X_test)

In [ ]:
mean_variance_mpo = MultiPeriodMeanVariance(
    num_assets=NUM_ASSETS,
    multi_period_step=multi_step_horizon,
    min_individual_allocation=0.001,
    max_individual_allocation=0.4,
    risk_aversion_coefficient=0.5,
    trade_cost=0.0025,
    holding_cost=0
)

In [ ]:
result_mv = []
for hist, mu, Sigma, pred in X_test:
   if len(pred) < multi_step_horizon:
      continue
   mv_opt_weights = mean_variance_mpo(mu, Sigma)
   portfolio_value = (mv_opt_weights*pred).sum(1).prod()
   result_mv.append(portfolio_value)

In [ ]:
result_mv = pd.Series(result_mv, index=test_df_cumulative_return[-len(result_mv):].index)
result_mv = result_mv.pct_change().fillna(0).cumsum()

In [ ]:
cac40_etf_ticker = "^FCHI"
cac40_index = yf.download(cac40_etf_ticker, 
                        start=start_date, 
                        end=end_date,
                        interval=data_interval)['Adj Close']
cac40_index = cac40_index.pct_change().fillna(0)

In [ ]:
cac40_index = cac40_index[result_mv.index]
cac40_index = np.cumsum(cac40_index)

In [ ]:
plt.plot(result_mv.index, result_mv, label='Mu-Var')
plt.plot(cac40_index.index, cac40_index, label='CAC-40')
plt.title('Out-of-Sample Test')
plt.xlabel("Date")
plt.ylabel("Return")
plt.legend()
plt.title("MPO - Benchmark")
plt.show()

In [ ]:
result_mv.name = "MV"
cac40_index.name = "CAC40"
output_filename = "performances_cac40"

In [ ]:
save_df = pd.DataFrame()
save_df["CAC40"] = cac40_index
save_df["MV"] = result_mv

In [ ]:
save_df.to_csv(f"data/processed/{output_filename}.csv")